In [1]:
import os
import sys
import json
import datetime
import wandb
import numpy as np
from sklearn.model_selection import train_test_split
from mmengine.config import Config
from mmengine.runner import Runner
from mmengine.hooks import EarlyStoppingHook
from mmengine.visualization import Visualizer, WandbVisBackend
from mmdet.utils import setup_cache_size_limit_of_dynamo
from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit

# mmdetection을 시스템 경로에 추가
sys.path.insert(0, "../mmdetection")


/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 사용자 맞춤형 학습 설정
config_path = './configs/grounding_dino/grounding_dino_swin-b_finetune_16xb2_1x_coco.py'  # DINO 설정 파일 경로
work_dir = './work_dirs/co_dino_custom'  # 로그와 모델을 저장할 디렉토리 경로
train_data_root = '/data/ephemeral/home/dataset/'  # 학습 데이터 경로
original_ann_file = '/data/ephemeral/home/dataset/train.json'  # 전체 데이터 어노테이션 파일 경로

In [3]:
# 클래스 설정
classes = (
    "General trash", "Paper", "Paper pack", "Metal", "Glass",
    "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing"
)
num_classes = len(classes)  # 10

In [4]:
# 학습 및 검증 데이터셋 나누기 (멀티라벨 스트라티파이드 분할)
with open(original_ann_file, 'r') as f:
    annotations = json.load(f)

# 이미지 ID 리스트 가져오기
image_ids = [image['id'] for image in annotations['images']]

# 이미지 ID와 해당 이미지에 포함된 클래스 목록 매핑
image_id_to_classes = {image_id: [] for image_id in image_ids}
for ann in annotations['annotations']:
    image_id = ann['image_id']
    category_id = ann['category_id']
    if category_id not in image_id_to_classes[image_id]:
        image_id_to_classes[image_id].append(category_id)

# 멀티라벨 인코딩을 위한 이진 매트릭스 생성
num_images = len(image_ids)
label_matrix = np.zeros((num_images, num_classes))
image_id_to_index = {image_id: idx for idx, image_id in enumerate(image_ids)}
for image_id, class_ids in image_id_to_classes.items():
    idx = image_id_to_index[image_id]
    for class_id in class_ids:
        label_matrix[idx, class_id - 1] = 1  # category_id는 1부터 시작

# 멀티라벨 스트라티파이드 셔플 스플릿 사용
msss = MultilabelStratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_indices, val_indices = next(msss.split(image_ids, label_matrix))

# 인덱스를 이미지 ID로 변환
train_ids = [image_ids[idx] for idx in train_indices]
val_ids = [image_ids[idx] for idx in val_indices]

# 학습 및 검증 데이터 어노테이션 생성
train_annotations = {
    'images': [img for img in annotations['images'] if img['id'] in train_ids],
    'annotations': [ann for ann in annotations['annotations'] if ann['image_id'] in train_ids],
    'categories': annotations['categories']
}
val_annotations = {
    'images': [img for img in annotations['images'] if img['id'] in val_ids],
    'annotations': [ann for ann in annotations['annotations'] if ann['image_id'] in val_ids],
    'categories': annotations['categories']
}

# 분할된 어노테이션을 파일로 저장
train_ann_file = '/data/ephemeral/home/dataset/train_split.json'
val_ann_file = '/data/ephemeral/home/dataset/val_split.json'

with open(train_ann_file, 'w') as f:
    json.dump(train_annotations, f)
with open(val_ann_file, 'w') as f:
    json.dump(val_annotations, f)

In [5]:
# 어노테이션 파일 수정: category_id를 1부터 시작하도록 조정
def increment_category_id(annotation_file):
    with open(annotation_file, 'r') as f:
        data = json.load(f)

    # 'categories' 섹션의 'id'를 1부터 시작하도록 수정
    id_mapping = {}
    for category in data['categories']:
        old_id = category['id']
        new_id = old_id + 1
        id_mapping[old_id] = new_id
        category['id'] = new_id

    # 'annotations' 섹션의 'category_id'를 매핑에 따라 수정
    for ann in data['annotations']:
        old_cat_id = ann['category_id']
        if old_cat_id in id_mapping:
            ann['category_id'] = id_mapping[old_cat_id]
        else:
            print(f"Warning: annotation id {ann['id']} has invalid category_id {old_cat_id}")

    # 수정된 데이터를 원래 파일에 저장
    with open(annotation_file, 'w') as f:
        json.dump(data, f, indent=4)

# 어노테이션 파일에 category_id를 1부터 시작하도록 수정
increment_category_id(train_ann_file)
increment_category_id(val_ann_file)

In [6]:
# 설정 파일 로드
cfg = Config.fromfile(config_path)

# 작업 디렉토리 수정
cfg.work_dir = work_dir  # 로그와 모델 저장을 위한 작업 디렉토리 수정

# backbone의 checkpoint 비활성화
cfg.model.backbone.with_cp = False

# encoder의 checkpoint 비활성화
cfg.model.encoder.num_cp = 0


# EarlyStoppingHook 추가
early_stopping_hook = dict(
    type='EarlyStoppingHook',
    monitor='bbox_mAP',
    rule='greater',
    min_delta=0.001,
    patience=5,
    check_finite=True,
    stopping_threshold=None
)

# cfg.default_hooks에 EarlyStoppingHook 추가
cfg.default_hooks.update(
    early_stopping=early_stopping_hook
)

# WandB 초기화 (Runner 전에)
run_name = f'grounding_dino_swin-b_finetune_16xb2_1x_coco_{datetime.datetime.now().strftime("%Y%m%d_%H%M%S")}'
wandb.init(
    project='Object_detection',
    name=run_name,
    config=cfg.to_dict(),
    allow_val_change=True,
    reinit=True
)
print(f"WandB Run Initialized: {run_name}")

# WandbVisBackend 설정
wandb_vis_backend = dict(
    type='WandbVisBackend',
    save_dir=cfg.work_dir,  # 저장할 디렉토리
    init_kwargs=dict(
        project='Object_detection',  # WandB 프로젝트 이름
        name=run_name,                # 고유한 실행 이름
        allow_val_change=True         # 설정 값 변경 허용
    ),
    define_metric_cfg=None,
    commit=True,
    log_code_name=None,
    watch_kwargs=None
)

# Visualizer 설정을 Runner의 visualizer 필드로 추가
cfg.visualizer = dict(
    type='Visualizer',
    vis_backends=[
        wandb_vis_backend  # WandbVisBackend 추가
    ],
    name='visualizer'  # Visualizer 이름 (옵션)
)

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


wandb: Currently logged in as: youngtae0818 (youngtae0818-naver). Use `wandb login --relogin` to force relogin


WandB Run Initialized: grounding_dino_swin-b_finetune_16xb2_1x_coco_20241018_003904


In [7]:
# 데이터셋 루트 경로 수정
cfg.train_dataloader.dataset.ann_file = train_ann_file  # 학습 데이터 어노테이션 파일 경로 수정
cfg.train_dataloader.dataset.data_prefix = dict(img=train_data_root)  # 학습 데이터 이미지 경로 수정

cfg.val_dataloader.dataset.ann_file = val_ann_file  # 검증 데이터 어노테이션 파일 경로 수정
cfg.val_dataloader.dataset.data_prefix = dict(img=train_data_root)  # 검증 데이터 이미지 경로 수정

# 클래스 설정을 데이터셋의 metainfo에 추가
cfg.train_dataloader.dataset.metainfo = dict(classes=classes)  # 학습 데이터셋 클래스 설정
cfg.val_dataloader.dataset.metainfo = dict(classes=classes)    # 검증 데이터셋 클래스 설정

# 데이터 파이프라인 수정: 마스크 관련 부분 제거 및 불필요한 단계 제거
def modify_pipeline(pipeline):
    new_pipeline = []
    for step in pipeline:
        if step['type'] == 'LoadAnnotations':
            step['with_bbox'] = True
            step['with_mask'] = False  # 마스크 관련 부분 제거
        if step['type'] == 'CopyPaste':
            continue  # CopyPaste 변환 제거
        new_pipeline.append(step)
    return new_pipeline

cfg.train_dataloader.dataset.pipeline = modify_pipeline(cfg.train_dataloader.dataset.pipeline)
cfg.val_dataloader.dataset.pipeline = modify_pipeline(cfg.val_dataloader.dataset.pipeline)

# 클래스 수 수정
# bbox_head가 리스트인 경우 모든 헤드에 대해 num_classes 설정
if isinstance(cfg.model.bbox_head, list):
    for head in cfg.model.bbox_head:
        head['num_classes'] = num_classes
else:
    cfg.model.bbox_head['num_classes'] = num_classes

# # roi_head 내의 bbox_head도 설정
# if isinstance(cfg.model.roi_head, list):
#     for roi_head in cfg.model.roi_head:
#         if 'bbox_head' in roi_head:
#             roi_head['bbox_head']['num_classes'] = num_classes
# else:
#     if 'bbox_head' in cfg.model.roi_head:
#         cfg.model.roi_head['bbox_head']['num_classes'] = num_classes

# # query_head도 num_classes 설정
# if isinstance(cfg.model.query_head, list):
#     for q_head in cfg.model.query_head:
#         q_head['num_classes'] = num_classes
# else:
#     cfg.model.query_head['num_classes'] = num_classes

In [8]:
# 추가: 모델의 모든 관련 헤드에 num_classes가 올바르게 설정되었는지 확인
def verify_num_classes(cfg, num_classes):
    # bbox_head
    if isinstance(cfg.model.bbox_head, list):
        for head in cfg.model.bbox_head:
            assert head.get('num_classes') == num_classes, "bbox_head num_classes 설정 오류"
    else:
        assert cfg.model.bbox_head.get('num_classes') == num_classes, "bbox_head num_classes 설정 오류"
    
    # # roi_head.bbox_head
    # if isinstance(cfg.model.roi_head, list):
    #     for roi_head in cfg.model.roi_head:
    #         if 'bbox_head' in roi_head:
    #             assert roi_head['bbox_head'].get('num_classes') == num_classes, "roi_head.bbox_head num_classes 설정 오류"
    # else:
    #     if 'bbox_head' in cfg.model.roi_head:
    #         assert cfg.model.roi_head['bbox_head'].get('num_classes') == num_classes, "roi_head.bbox_head num_classes 설정 오류"
    
    # # query_head
    # if isinstance(cfg.model.query_head, list):
    #     for q_head in cfg.model.query_head:
    #         assert q_head.get('num_classes') == num_classes, "query_head num_classes 설정 오류"
    # else:
    #     assert cfg.model.query_head.get('num_classes') == num_classes, "query_head num_classes 설정 오류"
    
    print("모든 관련 헤드에 대해 num_classes가 올바르게 설정되었습니다.")

verify_num_classes(cfg, num_classes)

모든 관련 헤드에 대해 num_classes가 올바르게 설정되었습니다.


In [9]:
# 자동 혼합 정밀도 학습 사용 설정
use_amp = False  # 자동 혼합 정밀도(AMP) 학습 사용 여부
if use_amp:
    cfg.optim_wrapper.type = 'AmpOptimWrapper'
    cfg.optim_wrapper.loss_scale = 'dynamic'

# 학습률 자동 스케일링 설정
auto_scale_lr = False  # 학습률 자동 스케일링 사용 여부
if auto_scale_lr:
    if 'auto_scale_lr' in cfg and 'enable' in cfg.auto_scale_lr and 'base_batch_size' in cfg.auto_scale_lr:
        cfg.auto_scale_lr.enable = True
    else:
        raise RuntimeError('설정 파일에 "auto_scale_lr" 또는 필요한 키가 없습니다.')


In [10]:
# 데이터 로더 설정 최적화
cfg.train_dataloader.batch_size = 4  # 배치 사이즈를 1로 줄임
cfg.val_dataloader.batch_size = 4

cfg.train_dataloader.num_workers = 2  # 워커 수 줄이기
cfg.val_dataloader.num_workers = 2

# Prefetch factor와 persistent_workers 설정
cfg.train_dataloader.prefetch_factor = 2
cfg.train_dataloader.persistent_workers = False
cfg.val_dataloader.prefetch_factor = 2
cfg.val_dataloader.persistent_workers = False

In [11]:
# 학습 재개 설정
resume_training = False  # 학습 재개 여부 설정 (True로 설정 시 학습 재개)
resume_checkpoint_path = './work_dirs/co_dino_custom/epoch_6.pth'  # 학습 재개 시 체크포인트 경로 지정
if resume_training:
    if resume_checkpoint_path is None:
        cfg.resume = True
        cfg.load_from = None
    else:
        cfg.resume = True
        cfg.load_from = resume_checkpoint_path

In [12]:
# 반복적인 컴파일 횟수를 줄여 학습 속도 향상
setup_cache_size_limit_of_dynamo()

In [13]:
# 평가 설정 추가 및 수정
cfg.evaluation = dict(interval=1, metric='bbox', save_best='auto')

# 평가자(evaluator) 설정 수정
if hasattr(cfg, 'val_evaluator'):
    # 평가자가 딕셔너리 또는 리스트로 정의되어 있을 경우
    if isinstance(cfg.val_evaluator, dict):
        cfg.val_evaluator.ann_file = val_ann_file
    elif isinstance(cfg.val_evaluator, list):
        for evaluator in cfg.val_evaluator:
            evaluator['ann_file'] = val_ann_file
elif 'evaluation' in cfg:
    # 평가 섹션 내에 ann_file을 추가
    cfg.evaluation['ann_file'] = val_ann_file


In [14]:
# 설정 파일을 통해 러너 생성
runner = Runner.from_cfg(cfg)

# ---------------------- 레이어 동결 및 Unfreeze 설정 시작 ----------------------

# 2. 모든 레이어를 동결
for name, param in runner.model.named_parameters():
    param.requires_grad = False

# 3. 필요한 레이어 Unfreeze

# bbox_head Unfreeze
for param in runner.model.bbox_head.parameters():
    param.requires_grad = True

# backbone의 특정 블록 Unfreeze (stages.3.blocks.0, stages.3.blocks.1)
for name, param in runner.model.backbone.named_parameters():
    if 'stages.3.blocks.0' in name or 'stages.3.blocks.1' in name:
        param.requires_grad = True

# neck Unfreeze
for param in runner.model.neck.parameters():
    param.requires_grad = True

# encoder Unfreeze
for param in runner.model.encoder.parameters():
    param.requires_grad = True

# decoder Unfreeze
for param in runner.model.decoder.parameters():
    param.requires_grad = True

# positional_encoding Unfreeze
for param in runner.model.positional_encoding.parameters():
    param.requires_grad = True

# 7. Optimizer 설정 수정
cfg.optim_wrapper.paramwise_cfg = dict(
    custom_keys={
        'backbone.stages.3.blocks.0': dict(lr_mult=0.1),
        'backbone.stages.3.blocks.1': dict(lr_mult=0.1),
        'neck': dict(lr_mult=1.0),
        'encoder': dict(lr_mult=1.0),
        'decoder': dict(lr_mult=1.0),
        'positional_encoding': dict(lr_mult=1.0),
        'bbox_head': dict(lr_mult=1.0)
    }
)

# ---------------------- 레이어 동결 및 Unfreeze 설정 끝 ----------------------

10/18 00:39:06 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.10.13 (main, Sep 11 2023, 13:44:35) [GCC 11.2.0]
    CUDA available: True
    numpy_random_seed: 792665384
    GPU 0: Tesla V100-SXM2-32GB
    CUDA_HOME: None
    GCC: gcc (Ubuntu 9.4.0-1ubuntu1~20.04.2) 9.4.0
    PyTorch: 1.12.1+cu116
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201402
  - Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191122 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.6.0 (Git Hash 52b5f107dd9cf10910aaa19cb47f3abf9b349815)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.6
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_70,code

TypeError: GroundingDinoTransformerEncoder.__init__() missing 2 required positional arguments: 'text_layer_cfg' and 'fusion_layer_cfg'

In [15]:
for name, param in runner.model.backbone.named_parameters():
    print(name)


patch_embed.projection.weight
patch_embed.projection.bias
patch_embed.norm.weight
patch_embed.norm.bias
stages.0.blocks.0.norm1.weight
stages.0.blocks.0.norm1.bias
stages.0.blocks.0.attn.w_msa.relative_position_bias_table
stages.0.blocks.0.attn.w_msa.qkv.weight
stages.0.blocks.0.attn.w_msa.qkv.bias
stages.0.blocks.0.attn.w_msa.proj.weight
stages.0.blocks.0.attn.w_msa.proj.bias
stages.0.blocks.0.norm2.weight
stages.0.blocks.0.norm2.bias
stages.0.blocks.0.ffn.layers.0.0.weight
stages.0.blocks.0.ffn.layers.0.0.bias
stages.0.blocks.0.ffn.layers.1.weight
stages.0.blocks.0.ffn.layers.1.bias
stages.0.blocks.1.norm1.weight
stages.0.blocks.1.norm1.bias
stages.0.blocks.1.attn.w_msa.relative_position_bias_table
stages.0.blocks.1.attn.w_msa.qkv.weight
stages.0.blocks.1.attn.w_msa.qkv.bias
stages.0.blocks.1.attn.w_msa.proj.weight
stages.0.blocks.1.attn.w_msa.proj.bias
stages.0.blocks.1.norm2.weight
stages.0.blocks.1.norm2.bias
stages.0.blocks.1.ffn.layers.0.0.weight
stages.0.blocks.1.ffn.layers.0.0

In [16]:
# 학습 가능한 파라미터 출력 (디버깅 용도)
trainable_params = [name for name, param in runner.model.named_parameters() if param.requires_grad]
print("Trainable parameters:")
for name in trainable_params:
    print(name)


Trainable parameters:
backbone.stages.3.blocks.0.norm1.weight
backbone.stages.3.blocks.0.norm1.bias
backbone.stages.3.blocks.0.attn.w_msa.relative_position_bias_table
backbone.stages.3.blocks.0.attn.w_msa.qkv.weight
backbone.stages.3.blocks.0.attn.w_msa.qkv.bias
backbone.stages.3.blocks.0.attn.w_msa.proj.weight
backbone.stages.3.blocks.0.attn.w_msa.proj.bias
backbone.stages.3.blocks.0.norm2.weight
backbone.stages.3.blocks.0.norm2.bias
backbone.stages.3.blocks.0.ffn.layers.0.0.weight
backbone.stages.3.blocks.0.ffn.layers.0.0.bias
backbone.stages.3.blocks.0.ffn.layers.1.weight
backbone.stages.3.blocks.0.ffn.layers.1.bias
backbone.stages.3.blocks.1.norm1.weight
backbone.stages.3.blocks.1.norm1.bias
backbone.stages.3.blocks.1.attn.w_msa.relative_position_bias_table
backbone.stages.3.blocks.1.attn.w_msa.qkv.weight
backbone.stages.3.blocks.1.attn.w_msa.qkv.bias
backbone.stages.3.blocks.1.attn.w_msa.proj.weight
backbone.stages.3.blocks.1.attn.w_msa.proj.bias
backbone.stages.3.blocks.1.norm2.w

In [17]:

# 학습 시작 전에 검증 어노테이션 파일 존재 여부 확인
if not os.path.exists(val_ann_file):
    raise FileNotFoundError(f'Validation annotation file not found: {val_ann_file}')

# 학습 시작
runner.train()

loading annotations into memory...
Done (t=0.06s)
creating index...
index created!


10/17 23:50:34 - mmengine - INFO - paramwise_options -- backbone.stages.3.blocks.0.norm1.weight:lr=1e-05
10/17 23:50:34 - mmengine - INFO - paramwise_options -- backbone.stages.3.blocks.0.norm1.weight:weight_decay=0.0001
10/17 23:50:34 - mmengine - INFO - paramwise_options -- backbone.stages.3.blocks.0.norm1.weight:lr_mult=0.1
10/17 23:50:34 - mmengine - INFO - paramwise_options -- backbone.stages.3.blocks.0.norm1.bias:lr=1e-05
10/17 23:50:34 - mmengine - INFO - paramwise_options -- backbone.stages.3.blocks.0.norm1.bias:weight_decay=0.0001
10/17 23:50:34 - mmengine - INFO - paramwise_options -- backbone.stages.3.blocks.0.norm1.bias:lr_mult=0.1
10/17 23:50:34 - mmengine - INFO - paramwise_options -- backbone.stages.3.blocks.0.attn.w_msa.relative_position_bias_table:lr=1e-05
10/17 23:50:34 - mmengine - INFO - paramwise_options -- backbone.stages.3.blocks.0.attn.w_msa.relative_position_bias_table:weight_decay=0.0001
10/17 23:50:34 - mmengine - INFO - paramwise_options -- backbone.stages.3

/data/ephemeral/home/baseline/mmdetection/mmdet/models/layers/positional_encoding.py:103: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  dim_t = self.temperature**(2 * (dim_t // 2) / self.num_feats)
/opt/conda/lib/python3.10/site-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
/data/ephemeral/home/baseline/mmdetection/mmdet/models/layers/transformer/utils.py:71: UserWarning: __floordiv__ is deprecated, and its behavior will change 

10/17 23:51:33 - mmengine - INFO - Epoch(train)  [1][ 50/983]  base_lr: 1.0000e-04 lr: 1.0000e-04  eta: 3:37:39  time: 1.1118  data_time: 0.0151  memory: 20773  grad_norm: 59.1553  loss: 16.2263  loss_cls: 0.7361  loss_bbox: 0.1572  loss_iou: 0.2117  d0.loss_cls: 0.7060  d0.loss_bbox: 0.1868  d0.loss_iou: 0.2441  d1.loss_cls: 0.7086  d1.loss_bbox: 0.1775  d1.loss_iou: 0.2328  d2.loss_cls: 0.7350  d2.loss_bbox: 0.1646  d2.loss_iou: 0.2190  d3.loss_cls: 0.7374  d3.loss_bbox: 0.1623  d3.loss_iou: 0.2150  d4.loss_cls: 0.7440  d4.loss_bbox: 0.1577  d4.loss_iou: 0.2115  enc_loss_cls: 0.7321  enc_loss_bbox: 0.2022  enc_loss_iou: 0.2668  dn_loss_cls: 0.5564  dn_loss_bbox: 0.3820  dn_loss_iou: 0.3702  d0.dn_loss_cls: 0.5924  d0.dn_loss_bbox: 0.5270  d0.dn_loss_iou: 0.5105  d1.dn_loss_cls: 0.5710  d1.dn_loss_bbox: 0.4040  d1.dn_loss_iou: 0.4076  d2.dn_loss_cls: 0.5603  d2.dn_loss_bbox: 0.3917  d2.dn_loss_iou: 0.3823  d3.dn_loss_cls: 0.5555  d3.dn_loss_bbox: 0.4029  d3.dn_loss_iou: 0.3857  d4.dn_

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/data/ephemeral/home/baseline/mmdetection/mmdet/models/layers/positional_encoding.py:103: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use 

10/18 00:09:15 - mmengine - INFO - Epoch(val)  [1][ 50/239]    eta: 0:01:39  time: 0.5273  data_time: 0.0117  memory: 20209  
10/18 00:09:41 - mmengine - INFO - Epoch(val)  [1][100/239]    eta: 0:01:13  time: 0.5231  data_time: 0.0076  memory: 3204  
10/18 00:10:07 - mmengine - INFO - Epoch(val)  [1][150/239]    eta: 0:00:46  time: 0.5218  data_time: 0.0067  memory: 3204  
10/18 00:10:33 - mmengine - INFO - Epoch(val)  [1][200/239]    eta: 0:00:20  time: 0.5202  data_time: 0.0064  memory: 3204  
10/18 00:10:58 - mmengine - INFO - Evaluating bbox...
Loading and preparing results...
DONE (t=1.14s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=14.65s).
Accumulating evaluation results...
DONE (t=7.19s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.524
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.607
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 

/opt/conda/lib/python3.10/site-packages/mmengine/hooks/early_stopping_hook.py:148: UserWarning: Skip early stopping process since the evaluation results (dict_keys(['coco/bbox_mAP', 'coco/bbox_mAP_50', 'coco/bbox_mAP_75', 'coco/bbox_mAP_s', 'coco/bbox_mAP_m', 'coco/bbox_mAP_l'])) do not include `monitor` (bbox_mAP).
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


10/18 00:11:41 - mmengine - INFO - Exp name: grounding_dino_swin-b_finetune_16xb2_1x_coco_20241017_235014
10/18 00:12:17 - mmengine - INFO - Epoch(train)  [2][ 50/983]  base_lr: 1.0000e-04 lr: 1.0000e-04  eta: 3:17:46  time: 1.0771  data_time: 0.0158  memory: 21480  grad_norm: 40.3115  loss: 9.6978  loss_cls: 0.4492  loss_bbox: 0.1346  loss_iou: 0.2251  d0.loss_cls: 0.4663  d0.loss_bbox: 0.1537  d0.loss_iou: 0.2458  d1.loss_cls: 0.4582  d1.loss_bbox: 0.1388  d1.loss_iou: 0.2307  d2.loss_cls: 0.4551  d2.loss_bbox: 0.1344  d2.loss_iou: 0.2266  d3.loss_cls: 0.4490  d3.loss_bbox: 0.1363  d3.loss_iou: 0.2270  d4.loss_cls: 0.4489  d4.loss_bbox: 0.1354  d4.loss_iou: 0.2256  enc_loss_cls: 0.4729  enc_loss_bbox: 0.1695  enc_loss_iou: 0.2731  dn_loss_cls: 0.0854  dn_loss_bbox: 0.1941  dn_loss_iou: 0.2637  d0.dn_loss_cls: 0.1750  d0.dn_loss_bbox: 0.3484  d0.dn_loss_iou: 0.4224  d1.dn_loss_cls: 0.1105  d1.dn_loss_bbox: 0.2389  d1.dn_loss_iou: 0.3174  d2.dn_loss_cls: 0.0933  d2.dn_loss_bbox: 0.2081

KeyboardInterrupt: 